In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
from gives import *
df_phone.head()

In [ ]:
df=df_agg(df_phone,freq='H',y_name='volume')

In [ ]:
# df=df_phone.copy()
# # Agréger les vases par jour et compter le nombre de cases pour chaque jour
# df = df.groupby(pd.Grouper(key='Consideration Start', freq='H')).size().reset_index(name='volume')

# # Ajout des features temporelles détaillées
# df=df.assign(year=df['Consideration Start'].dt.isocalendar()['year'], 
#              week=df['Consideration Start'].dt.isocalendar()['week'],
#              day=df['Consideration Start'].dt.isocalendar()['day'],
#              #quarter=df['Consideration Start'].dt.quarter,
#              month=df['Consideration Start'].dt.month,
#              day_mon=df['Consideration Start'].dt.day,
#              hour=df['Consideration Start'].dt.hour*100+df['Consideration Start'].dt.minute)

# # Éjection des H off-office lorsque l'on est freq=min
# df.drop(index=df.loc[(df['hour']<800) | (df['hour']>=1800)].index, inplace=True)

# # Conversion de la colonne de date en index
# df.set_index('Consideration Start', inplace=True)

# # Éjection des samedis et dimanches lorsque l'on est en phone only
# df.drop(index=df.loc[(df['day']==6) | (df['day']==7)].index, inplace=True)

# # Conversion dans le type adéquat pour LGBM
# df.loc[:,df.columns] = df.loc[:,df.columns].astype(int)

# # Rajoute les vacances, jours fériés et d-1 fériés
# df=df.assign(vac_A=df.apply(lambda x : d.is_holiday_for_zone(datetime.date(x['year'],x['month'],x['day_mon']),'A'),axis=1),
#              vac_B=df.apply(lambda x : d.is_holiday_for_zone(datetime.date(x['year'],x['month'],x['day_mon']),'B'),axis=1),
#              vac_C=df.apply(lambda x : d.is_holiday_for_zone(datetime.date(x['year'],x['month'],x['day_mon']),'C'),axis=1),
#              ouvre=df.apply(lambda x : cal.is_working_day(datetime.date(x['year'],x['month'],x['day_mon'])),axis=1),
#              begin=df.apply(lambda x : 800<=x['hour']<900, axis=1),
#              lunch=df.apply(lambda x : 1200<=x['hour']<1300, axis=1),
#              end=df.apply(lambda x : 1700<=x['hour']<1800, axis=1)
#              )


# df['d-1_fer']=((df['ouvre'].shift(1,fill_value=True)-1)*-1).apply(bool)
# df.loc[(df['day']==1) & (df['d-1_fer'])==True,'d-1_fer']=False 

# # Éjection des samedis et dimanches lorsque l'on est en phone only ainsi que les jours fériés
# df.drop(index=df.loc[df.ouvre==False].index, inplace=True)
# df.drop(columns='ouvre', inplace=True)

In [ ]:
df.head(40)

In [ ]:
# plt.figure(figsize=((10,10)))
# corr=df.corr()
# sns.heatmap(corr, vmax=1.0, center=0, fmt='.2f', square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
# plt.show()

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(df, train_beg='2020-04-01', train_end='2022-03-31', test_beg='2022-04-01', test_end='2023-04-12', y_name='volume')

In [ ]:
# #Données train et test
# train=df.loc['2020-04-01':'2022-03-31']
# test=df.loc['2022-04-01':'2023-04-12']

# # Variables d'entrée
# X_train = train.drop(['volume'], axis=1)
# X_test = test.drop(['volume'], axis=1)

# # Variable de sortie
# y_train = train['volume']
# y_test = test['volume']

In [ ]:
model = lgb.LGBMRegressor(objective= 'tweedie',
    num_leaves= 31,
    learning_rate= 0.2,
                         )
model.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=0)
lgb.plot_metric(model)

In [ ]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    #'cat_feature=name' : ['year', 'week', 'day','quarter', 'month', 'day_mon', 'vac_A', 'vac_B', 'vac_C', 'lunch', 'd-1_fer']
}

model_bis = lgb.LGBMRegressor(**params)
model_bis.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=0)
lgb.plot_metric(model_bis)

In [ ]:
# train_lgb = lgb.Dataset(data = X_train, label = y_train, feature_name = list(X_train))
# cv_results = lgb.cv(params,train_lgb,stratified=False, shuffle=False, num_boost_round=1)

In [ ]:
tabl_err([model, model_bis], X_test, y_test)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(name="Vraies valeurs", x=y_test.index, y=y_test, mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM", x=y_test.index, y=model.predict(X_test).astype(int), mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM_bis", x=y_test.index, y=model_bis.predict(X_test).astype(int), mode='lines+markers'))
fig.show()

# avec boucle for :
# fig = go.Figure()
# for y_ in [test['volume'],model.predict(X_test).astype(int),model_bis.predict(X_test).astype(int)] :
#     fig.add_trace(go.Scatter(name="Vraies valeurs", x=test.index, y=y_, mode='lines+markers'))
# fig.show()

In [ ]:
y_pred=model_bis.predict(X_test).astype(int)
# np.linalg.norm(pic(y_test)[0]-pic(y_pred)[0],1)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(name="Vraies valeurs", x=y_test.index, y=y_test, mode='lines+markers'))
fig.add_trace(go.Scatter(name="Pics", x=y_test[pic(y_test)[0]].index, y=y_test[pic(y_test)[0]], mode='markers', marker_color='green'))
fig.show()


## Learning error

In [ ]:
tabl_err([model, model_bis], train.loc['2021-03-08':'2022-03-08'].drop(['volume'], axis=1), df.loc['2021-03-08':'2022-03-08'].volume)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name="Vraies valeurs", x=train.loc['2021-03-08':'2022-03-08'].index, y=df.loc['2021-03-08':'2022-03-08'].volume, mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM", x=train.loc['2021-03-08':'2022-03-08'].index, y=model.predict(train.loc['2021-03-08':'2022-03-08'].drop(['volume'], axis=1)).astype(int), mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM_bis", x=train.loc['2021-03-08':'2022-03-08'].index, y=model_bis.predict(train.loc['2021-03-08':'2022-03-08'].drop(['volume'], axis=1)).astype(int), mode='lines+markers'))
fig.show()

# avec boucle for :
# fig = go.Figure()
# for y_ in [test['volume'],model.predict(X_test).astype(int),model_bis.predict(X_test).astype(int)] :
#     fig.add_trace(go.Scatter(name="Vraies valeurs", x=test.index, y=y_, mode='lines+markers'))
# fig.show()

## Plot Aver. Perc. Err.

In [ ]:
fig = go.Figure()
test_error=predict_test(model, pd.DataFrame(), X_test, y_test)
fig.add_trace(go.Scatter(name="LGBM", x=test.index, y=test_error["absolute_error_%"], mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM_bis", x=test.index, y=predict_test(model_bis, pd.DataFrame(), X_test, y_test)["absolute_error_%"], mode='lines+markers'))

## Learning & Test Error

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name="Vraies valeurs", x=df.index, y=pd.concat([df.loc[:'2022-03-08'].volume,test.volume]), mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM", x=train.loc[:'2022-03-08'].index, y=model.predict(train.loc[:'2022-03-08'].drop(['volume'], axis=1)).astype(int), mode='lines+markers'))
fig.add_trace(go.Scatter(name="LGBM_bis", x=train.loc[:'2022-03-08'].index, y=model_bis.predict(train.loc[:'2022-03-08'].drop(['volume'], axis=1)).astype(int), mode='lines+markers'))

fig.add_trace(go.Scatter(name="LGBM_test", x=test.index, y=model.predict(X_test).astype(int), mode='lines+markers', marker_color='violet'))
fig.add_trace(go.Scatter(name="LGBM_bis_test", x=test.index, y=model_bis.predict(X_test).astype(int), mode='lines+markers',  marker_color='yellow'))
fig.show()

In [ ]:
tabl_err_all([model, model_bis], train.loc['2021-03-08':'2022-03-08'].drop(['volume'], axis=1), df.loc['2021-03-08':'2022-03-08'].volume, X_test, y_test) 